[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dimitarpg13/langchain_tutorial/blob/main/langchain_tutorial/notebooks/summarization/SimpleExample.ipynb)


### Use Case

Suppose you have a set of documents (PDFs, Notion pages, customer questions, etc) and you want to summarize the content.
A central question for building a summarizer is how to pass your documents into the LLM's context window. Two common approaches for this are:

1. `Stuff`: simply _stuff_ all your documents into a single promnpt. This is the simplest approach.
2. `Map-reduce`: Summarize each document on its own in a _map_ step and then _reduce_ the summaries into a final summary


In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [2]:
import os
from dotenv import load_dotenv

os.environ["LANGSMITH_TRACING"] = "true"
load_dotenv()

True

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

In [4]:
pip install -qU "langchain[openai]"

In [5]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

## Stuff: summarize in a single LLM call

We can use [create_stuff_documents_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html), especially if using models with larger context window (that is, at least hundred of kilobytes).

Examples of models with large context windows:

* 128K token OpenAI `gpt-4o`
* 200K token Anthropic `claude-3-5-sonnet-20240620`

The chain will take a list of documents, insert them all into a prompt , and pass that prompt to an LLM:


In [6]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate

# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

# Instantiate chain
chain = create_stuff_documents_chain(llm, prompt)

# Invoke chain
result = chain.invoke({"context": docs})
print(result)

The article "LLM Powered Autonomous Agents" by Lilian Weng provides an in-depth exploration of autonomous agents that leverage large language models (LLMs) as their central controllers. It outlines a comprehensive system architecture for these agents, highlighting three main components: planning, memory, and tool usage.

1. **Planning:** This component involves task decomposition into manageable subtasks and self-reflection, allowing agents to learn from past experiences and improve their decision-making processes. Various methodologies, such as Chain of Thought (CoT) and Tree of Thoughts, are discussed for enhancing planning capabilities.

2. **Memory:** The article covers different types of memory, including sensory, short-term, and long-term memory, which help agents retain and recall information. Techniques for maximum inner product search (MIPS) are introduced to facilitate efficient information retrieval.

3. **Tool Use:** The ability to interact with external tools significantly

## Streaming
Note that we can also stream the result token-by-token:

In [7]:
for token in chain.stream({"context": docs}):
    print(token, end="|")

|The| article| "|LL|M| Powered| Autonomous| Agents|"| by| Lil|ian| W|eng| discusses| the| potential| of| large| language| models| (|LL|Ms|)| as| the| core| controllers| of| autonomous| agents|.| It| presents| an| overview| of| an| L|LM|-powered| autonomous| agent| system|,| which| includes| three| main| components|:| Planning|,| Memory|,| and| Tool| Use|.| 

|1|.| **|Planning|**|:| This| involves| task| decomposition| into| manageable| sub|go|als| and| self|-ref|lection| to| improve| the| agent|'s| actions| based| on| past| experiences|.| Techniques| such| as| Chain| of| Thought| (|Co|T|)| and| Tree| of| Thoughts| (|To|T|)| are| highlighted| for| planning| complex| tasks|.

|2|.| **|Memory|**|:| The| system| utilizes| various| memory| types| —| including| short|-term| and| long|-term| memory| —| enabling| agents| to| retain| significant| information|.| Maximum| Inner| Product| Search| (|M|IPS|)| algorithms| are| employed| for| efficient| information| retrieval| from| memory|.

|3|.| **